In [61]:
import os
import copy
import numpy as np
from CT import construct_CT_object
import os
import json
import glob
import random


In [62]:

def get_path_structures_DICOM_file(path_CT_DICOM_series):
    filenames = [filename for filename in os.listdir(path_CT_DICOM_series) if filename.startswith('RS')]
    print(filenames)

    assert len(filenames) > 0, "no structures DICOM file found"
    assert len(filenames) <= 1, "more than one structures DICOM files found"

    return os.path.join(path_CT_DICOM_series, filenames[0])



In [63]:


# root_patient = "/data/oosterhoff/patients/DBP_OP007/"
# name_fixed_CT = 'pCTp0'
# name_moving_CT = 'rCTp12'

# path_fixed_CT = os.path.join(root_patient, name_fixed_CT, '')
# path_moving_CT = os.path.join(root_patient, name_moving_CT, '')


# path_struct_fixed_CT = get_path_structures_DICOM_file(path_fixed_CT)
# path_struct_moving_CT = get_path_structures_DICOM_file(path_moving_CT)



# fixed_CT = construct_CT_object(name_fixed_CT, path_fixed_CT, path_struct_fixed_CT, roi_names = ['External'])
# moving_CT = construct_CT_object(name_moving_CT, path_moving_CT, path_struct_moving_CT, roi_names = ['External'])


In [64]:
# import matplotlib.pyplot as plt
# import SimpleITK as sitk
# fixed_CT_array = sitk.GetArrayFromImage(fixed_CT.image)
# moving_CT_array = sitk.GetArrayFromImage(moving_CT.image)
# plt.imshow(fixed_CT_array[80])

In [65]:
class CaseDataReader:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            self.data = json.load(file)
        
        self.id = self.data['patient_id']
        
        self.examinations_transformations = self.extract_details()


    def new_transfer_param(self, rigid_matrix):
        max_translation = 1.5
        matrix = rigid_matrix
        final_translation_coordinate = {'x': 0, 'y': 0, 'z': 0}  # Initialize the dictionary
        
        for idx, key in zip([3, 7, 11], ['x', 'y', 'z']):
            random_translation = (2 * random.random() - 1) * max_translation
            # random_translation = 0
            matrix[idx] = (matrix[idx] + random_translation)* 10 # change to mm
            final_translation_coordinate[key] = random_translation * 10 # Store the translation in mm
        
        # matrix = np.array(matrix).reshape(4, 4)
    
        return matrix, final_translation_coordinate

    def extract_details(self):
        examinations_transformations = []

        # Interested only in the examination part of the JSON file
        for exam_name, exam_details in self.data['examinations'].items():
            if 'pCT' not in exam_name:
                exam_dict = {'exam_name': exam_name, 'plans': []}  # Initialize with an empty plans list
                reg_to_planning = exam_details.get('registration_to_planning_examinations', {})

                for plan_name, registration_details in reg_to_planning.items():

                    # if 'F' not in plan_name:
                    if 'P' in plan_name:
                        from_exam = registration_details.get('from_examination_name')
                        to_exam = registration_details.get('to_examination_name')
                        rigid_matrix = registration_details.get('rigid_transformation_matrix', [])
                        print(rigid_matrix)

                        if rigid_matrix:
                            new_matrix, final_translation_coordinate = self.new_transfer_param(rigid_matrix)
                            print(new_matrix)
                            
                            print(final_translation_coordinate)
                            plan_dict = {
                                'plan_name': plan_name,
                                'planning_exam_name': to_exam,
                                'repeatedCT_name': from_exam,
                                'transformation_matrix': new_matrix,
                                'final_translation_coordinate': final_translation_coordinate

                            }
                            exam_dict['plans'].append(plan_dict)  # Add this plan to the exam's plans list

                if exam_dict['plans']:  # Only append if there are any plans
                    examinations_transformations.append(exam_dict)

        return examinations_transformations


    def get_plan_details(self):
        return self.examinations_transformations
    

    def save_to_json(self, output_file):
        data_to_save = {
        "id": self.id,
        "examination_details": self.examinations_transformations
    }
        with open(output_file, 'w') as file:
            json.dump(data_to_save, file, indent=4)

# # Usage example
# case_data_path = "/home/shahpouriz/Data/DBP_CTs/DBP_OP011_case_data.json"
# patient = CaseDataReader(case_data_path)

# examinations_transformations = patient.get_plan_details()
# patient.save_to_json('/home/shahpouriz/Data/DBP_CTs/DBP_OP011_coordination_data.json')


In [66]:
# Define the root directory to search for case_data.json files
case_data_root = "/home/shahpouriz/Data/DBP_CTs/"

# Use glob to find all *case_data.json files in the directory and its subdirectories
case_data_files = glob.glob(os.path.join(case_data_root, '**', '*case_data.json'), recursive=True)

for case_data_path in case_data_files:
    patient = CaseDataReader(case_data_path)

    examinations_transformations = patient.get_plan_details()
    
    # Generate the new file name based on the original file name
    # Here, we replace '_case_data.json' with '_coordination_data.json'
    new_file_name = case_data_path.replace('_case_data.json', '_coordination_data.json')
    
    # Optionally, ensure the directory exists or create it
    output_directory = os.path.dirname(new_file_name)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)
    
    # Save the processed data to the new file
    patient.save_to_json(new_file_name)


[0.9986988730599623, -0.022199807737857846, -0.045910015085638835, 0.6230528137061262, 0.021983917611558185, 0.9997447902067917, -0.005202097733230838, -12.029204660431501, 0.04601378396969391, 0.004186047154537345, 0.9989320340713929, 0.6494109629482141, 0.0, 0.0, 0.0, 1.0]
[0.9986988730599623, -0.022199807737857846, -0.045910015085638835, 21.04322035211065, 0.021983917611558185, 0.9997447902067917, -0.005202097733230838, -118.52226950158025, 0.04601378396969391, 0.004186047154537345, 0.9989320340713929, -5.3260823750188235, 0.0, 0.0, 0.0, 1.0]
{'x': 14.812692215049388, 'y': 1.7697771027347664, 'z': -11.820192004500964}
[0.9996932272771138, -0.017425377087747246, -0.01760135135774108, 0.3075104583319768, 0.017166396847591525, 0.9997437087603849, -0.014759119665546093, -12.048278833306778, 0.017854023511238028, 0.014452440187758013, 0.9997361455989673, 0.2846702770448578, 0.0, 0.0, 0.0, 1.0]
[0.9996932272771138, -0.017425377087747246, -0.01760135135774108, 0.40306697996517926, 0.017166

In [67]:
import warnings
import matplotlib.pyplot as plt
import SimpleITK as sitk

def process_examination_transformations(examinations_transformations, root_patient, saving_root_patient):
    """
    Process examination transformations for a given patient.

    Parameters:
    - examinations_transformations: List of examination transformations loaded from a JSON file.
    - root_patient: Base directory path where patient's DICOM files are located.
    - saving_root_patient: Directory path where processed data will be saved.
    """

    for item in examinations_transformations:
        exam_name = item.get('exam_name')
        for plan in item.get('plans', []):
            plan_name = plan.get('plan_name')
            moving_CT = plan.get('repeatedCT_name')
            fixed_CT = plan.get('planning_exam_name')
            transformation_matrix = np.array(plan.get('transformation_matrix')).reshape(4, 4)
            
            
            print(f"Exam: {exam_name}, Plan: {plan_name}")
            print(f"Moving CT: {moving_CT}, Fixed (Planning) CT: {fixed_CT}")
            print(f"Transformation Matrix: {plan.get('transformation_matrix')}")
            print("--------------------------------------------------")
            
            path_fixed_CT = os.path.join(root_patient, fixed_CT, '')
            path_moving_CT = os.path.join(root_patient, moving_CT, '')

            if not os.path.exists(path_fixed_CT) or not os.path.exists(path_moving_CT):
                warnings.warn(f"Path does not exist: {path_fixed_CT if not os.path.exists(path_fixed_CT) else path_moving_CT}")
                continue  # Skip th

            path_struct_fixed_CT = get_path_structures_DICOM_file(path_fixed_CT)
            path_struct_moving_CT = get_path_structures_DICOM_file(path_moving_CT)

            fixed_CT_obj = construct_CT_object(fixed_CT, path_fixed_CT, path_struct_fixed_CT, roi_names=['External'])
            moving_CT_obj = construct_CT_object(moving_CT, path_moving_CT, path_struct_moving_CT, roi_names=['External'])

            moving_CT_obj.transform(transformation_matrix, fixed_CT_obj)
            

            fixed_CT_obj.override_air_outside_external()
            moving_CT_obj.override_air_outside_external()

            fixed_ct_saving_path = os.path.join(saving_root_patient, plan_name, fixed_CT_obj.name)
            moving_ct_saving_path = os.path.join(saving_root_patient, plan_name, moving_CT_obj.name)

            plan['planningCT_filename'] = fixed_ct_saving_path
            plan['repeatedCT_filename'] = moving_ct_saving_path


            # fixed_CT_array = sitk.GetArrayFromImage(fixed_CT_obj.image)
            # moving_CT_array = sitk.GetArrayFromImage(moving_CT_obj.image)
            # plt.imshow(fixed_CT_array[80]-moving_CT_array[80])
            
            if not os.path.exists(fixed_ct_saving_path):
                os.makedirs(fixed_ct_saving_path, exist_ok=True)
                fixed_CT_obj.save(fixed_ct_saving_path, save_struct_file=False)

            os.makedirs(moving_ct_saving_path, exist_ok=True)
            moving_CT_obj.save(moving_ct_saving_path, save_struct_file=False)

    

In [68]:
import glob
import os
import json
import numpy as np

# Define the directory where the coordination JSON files are saved
coordination_json_root = "/home/shahpouriz/Data/DBP_CTs/"

# Use glob to find all *_coordination_data.json files in the directory
coordination_json_files = glob.glob(os.path.join(coordination_json_root, '**', '*_coordination_data.json'), recursive=True)

for json_file in coordination_json_files:
    # Load the JSON content
    with open(json_file, 'r') as file:
        data = json.load(file)


    patient_id = data["id"]  # Assuming the patient ID is stored at the top level
    examinations_transformations = data["examination_details"]  # The list of transformations
    root_patient = os.path.join("/data/oosterhoff/patients/", patient_id)
    saving_root_patient = os.path.join(coordination_json_root, patient_id)
    
    process_examination_transformations(examinations_transformations, root_patient, saving_root_patient)


    with open(json_file, 'w') as file:
        json.dump(data, file, indent=4)


Exam: rCTp1, Plan: A1PHH
Moving CT: rCTp1, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.9993826820268941, -0.008993967859802459, -0.0339612044378422, -8.284390460022337, 0.00874428626416786, 0.99993369137632, -0.0074933509364491935, -126.53707537983975, 0.034026347474602064, 0.007191758662756913, 0.999395060166336, 0.7812038551567552, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240308110022660.1168.17333.dcm']
['RS1.2.752.243.1.1.20240308110022664.1174.34061.dcm']


Exam: rCTp4, Plan: A1PHH
Moving CT: rCTp4, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.9994857934874054, -0.0013117561235219107, -0.03203791367620633, 10.584362151785784, 0.0010148283802517658, 0.9999564015776361, -0.009282514059452927, -120.93281225206765, 0.03204869326837338, 0.009245227946227634, 0.9994435486909764, -6.5626832463761975, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240308110022660.1168.17333.dcm']
['RS1.2.752.243.1.1.20240308110022664.1173.58260.dcm']
Exam: rCTp9, Plan: A1PHH
Moving CT: rCTp9, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.9994289905169145, -0.008331569192821476, -0.03274565420214676, 0.5236227855950043, 0.00837423777739526, 0.9999642555403959, 0.0011660974183629012, -119.73847750327691, 0.03273476830510628, -0.0014396514601439543, 0.9994630370092153, -15.599141485891385, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240308110022660.1168.1733

/tmp/ipykernel_1001471/1879756819.py:33: UserWarning: Path does not exist: /data/oosterhoff/patients/DBP_OP038/pCTp0/
  warnings.warn(f"Path does not exist: {path_fixed_CT if not os.path.exists(path_fixed_CT) else path_moving_CT}")
/tmp/ipykernel_1001471/1879756819.py:33: UserWarning: Path does not exist: /data/oosterhoff/patients/DBP_OP038/rCTp11/
  warnings.warn(f"Path does not exist: {path_fixed_CT if not os.path.exists(path_fixed_CT) else path_moving_CT}")
/tmp/ipykernel_1001471/1879756819.py:33: UserWarning: Path does not exist: /data/oosterhoff/patients/DBP_OP038/rCTp23/
  warnings.warn(f"Path does not exist: {path_fixed_CT if not os.path.exists(path_fixed_CT) else path_moving_CT}")


Exam: rCTp3, Plan: AAPHH
Moving CT: rCTp3, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.9995991397246103, 0.00019813404516099567, -0.028311139233864625, 12.340109290002918, -0.00021061182297525544, 0.9999998820061002, -0.0004377561488124725, -119.46993165034628, 0.028311049158926335, 0.0004435433304066565, 0.9995990635073819, 5.3084646084314455, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240306123218526.1594.66061.dcm']
['RS1.2.752.243.1.1.20240306123218527.1596.55517.dcm']
Exam: rCTp7, Plan: A1PHH
Moving CT: rCTp7, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.9997758932700996, 0.018982266196412013, -0.009372129215004599, -0.8525157585802001, -0.019010309781828962, 0.9998150463248305, -0.0029122610590697475, -130.75004891173265, 0.009315114490605413, 0.00308977548145975, 0.9999518398050483, 48.54555369441209, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240306123218526.1594

/tmp/ipykernel_1001471/1879756819.py:33: UserWarning: Path does not exist: /data/oosterhoff/patients/DBP_OP068/pCtp0/
  warnings.warn(f"Path does not exist: {path_fixed_CT if not os.path.exists(path_fixed_CT) else path_moving_CT}")


...

Exam: rCTp3, Plan: AAPHH
Moving CT: rCTp3, Fixed (Planning) CT: rCTp14
Transformation Matrix: [0.9999678160586898, 0.0015766961985402336, -0.007866439849891773, -4.122127056069855, -0.0015883722787608616, 0.9999976459023181, -0.0014782636185104395, 2.5781111022866723, 0.007864090558896213, 0.00149071087715101, 0.9999679664173057, 13.17931242932981, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240306140822531.1344.37135.dcm']
['RS1.2.752.243.1.1.20240306140822530.1342.47520.dcm']
Exam: rCTp9, Plan: A1PHH
Moving CT: rCTp9, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.9999477085518287, 0.006281634155530114, -0.008069772876791793, 0.12985796631738378, -0.006327567757648794, 0.9999638536840341, -0.005679191108587779, -111.41800177855659, 0.008033806583388127, 0.005729956170127006, 0.999951311591754, -5.0515201195712, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240306140822528.1340.63126.

/tmp/ipykernel_1001471/1879756819.py:33: UserWarning: Path does not exist: /data/oosterhoff/patients/DBP_OP069/pCTp0/
  warnings.warn(f"Path does not exist: {path_fixed_CT if not os.path.exists(path_fixed_CT) else path_moving_CT}")


Exam: rCTp3, Plan: AAPHH
Moving CT: rCTp3, Fixed (Planning) CT: rCTp9
Transformation Matrix: [0.9999625609225948, 0.006871218394013412, 0.005259573262842238, -11.221534287803394, -0.0069025045409489075, 0.9999584544058108, 0.00595356140504788, 9.09899149371219, -0.00521844653010959, -0.005989642737531824, 0.9999684455000012, 3.686370441249618, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240314132210988.1256.85250.dcm']
['RS1.2.752.243.1.1.20240314132210987.1255.73374.dcm']
Exam: rCTp9, Plan: A1PHH
Moving CT: rCTp9, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.9991198263185994, 0.002178267262046937, -0.04189066493655743, 20.36395059461728, -0.0038357675822418484, 0.9992111118399212, -0.0395277227105683, -120.0790742817579, 0.04177151594264745, 0.03965361430391496, 0.9983399878440659, 15.429656536894612, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20240314132210984.1253.68674.dcm']
['R

/tmp/ipykernel_1001471/1879756819.py:33: UserWarning: Path does not exist: /data/oosterhoff/patients/DBP_OP005/pCTp0/
  warnings.warn(f"Path does not exist: {path_fixed_CT if not os.path.exists(path_fixed_CT) else path_moving_CT}")


Exam: rCTp6, Plan: A1PHH
Moving CT: rCTp6, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.999969042494892, -0.004823707186705856, -0.006216582728918856, 13.69980854270615, 0.0048269319861921345, 0.999988223365972, 0.0005038422048638206, -113.30394523792714, 0.006214079131234586, -0.000533833629185218, 0.999980549931951, 10.828692861104239, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20231017113037180.1020.47342.dcm']
['RS1.2.752.243.1.1.20231017113037180.1018.54142.dcm']
Exam: rCTp11, Plan: A1PHH
Moving CT: rCTp11, Fixed (Planning) CT: pCTp0
Transformation Matrix: [0.9999796409750341, 0.005091423878956164, 0.0038464318955158867, -7.5921227844643315, -0.00509066041888768, 0.9999870208200431, -0.00020824974054962095, -127.75751174479909, -0.003847442259685943, 0.0001886646221835041, 0.9999925807693368, -12.432988622344293, 0.0, 0.0, 0.0, 1.0]
--------------------------------------------------
['RS1.2.752.243.1.1.20231017113037180.1020.